In [1]:
def process_train():
    """Function to process the data"""
    
    # Specify the root directory where your MLOps project is located
    root_directory = "C:/Github/MLOPs_WebApp/MLOps_Assignment"

    # Construct the absolute path to the configuration directory
    config_dir = os.path.normpath(os.path.join(root_directory, "config", "process"))

    # Initialize the Hydra config directory
    initialize_config_dir(config_dir=config_dir)

    # Load the Hydra configuration
    config = compose(config_name="process_Shermaine")
    print('All parameters in process_Shermaine.yaml: ' + str(config))

    # Load raw data
    medical = pd.read_csv(config.data.raw)

    # Filter to above 0.2 resting BP
    medical_clean = medical[medical['resting_BP'] > 0.2]

    # Perform PyCaret setup
    medical_setup = setup(data = medical_clean, 
                          target = config.setup.target, 
                          normalize = config.setup.norm, 
                          normalize_method=config.setup.norm_mtd, 
                          train_size = config.setup.trainsize, 
                          remove_outliers=config.setup.rmoutlier, 
                          bin_numeric_features=config.setup.bin,
                          session_id=config.setup.session, 
                          log_experiment=config.setup.logexp, 
                          experiment_name=config.setup.expname, 
                          fold=config.setup.fold
                         )
    
    # Print features information
    print(f'Numeric features: {medical_setup._fxs["Numeric"]}')
    print(f'Categorical features: {medical_setup._fxs["Categorical"]}')
    print(f'Ordinal features: {medical_setup._fxs["Ordinal"]}')

    # Get transformed dataset
    df = medical_setup.get_config('dataset_transformed')
    # print('Processed Data: ', df)

    # Save processed data
    file_name = 'medical_processed.csv'
    full_file = os.path.join(config.data.ppath, file_name)
    df.to_csv(full_file, index=False)
    
    print(f'Data was processed and written at {full_file}.')

    # Compare models (excluding specified models)
    best = compare_models()
    best = create_model('et')   #why et
    print(best)

    # Tune models
    tuned_best = tune_model(best, return_tuner=True) 
    best_tuned_model = tuned_best[0]  # Extract the best tuned model from the tuple
    print(best_tuned_model)

    # predict on test set
    best_pred = predict_model(best_tuned_model)
    print(best_pred)
    
    evaluate_model(best_tuned_model)

process_train()

NameError: name 'os' is not defined

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pycaret.regression import *

import mlflow
import mlflow.sklearn
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import os
import hydra
from omegaconf import DictConfig
from hydra.experimental import compose, initialize_config_dir